# **Introduction**

Nous avons travaillé sur les méthodes proposées dans l'article *Dimension Independent Matrix Square using MapReduce (DIMSUM)*.

L'article développe une solution générale pour pour calculer le produit $A^TA$ avec $A$ une matrice sparse de taille $n \times m$ avec $m >> n$.  On considère typiquement des matrices de grande taille avec $m \approx 10^{13}$ et $n \approx 10^4$. L'algorithme proposé fait appel à la méthode MapReduce pour paralléliser les calculs.

La plupart des méthodes de calculs de $A^TA$ nécessitent d'avoir accès à la totalité de la matrice $A$ sur une seule machine ou bien d'échanger un grand nombre d'informations entre les machines, ce qui n'est pas faisable pour les valeurs de $m$ considérées. Une matrice de cette taille ne peut être stockée ou streamée sur une seule machine, ce qui nécessite d'utiliser un algorithme adapté.

MapReduce est utilisé pour gérer une grande quantité de données (ici la matrice de taille $m \times n$) en distribuant le calcul sur plusieurs machines. L'algorithme présenté se base sur la décomposition en valeurs singulières d'une matrice de taille $m \times n$ avec au plus $L$ valeurs non-nulles par ligne (sparsité). La décomposition en valeurs singulirères (SVD) de $A$ s'écrit: $A = UΣV^T$, avec $U$ et $V$ deux matrices unitaires de tailles $m \times m$ et $n \times n$. L'article développe un algorithme permettant de calculer le produit $A^TA$ sans dépendance à $m$ et fournit une borne supérieure pour l'erreur.

**Objectif:** Calculer le produit $A^TA$ avec $A$ une matrice $m \times n$  ($m >> n$)

# **I. Algorithmes et implémentations**

*(Les codes proposés dans cette parties ne sont pas utilisables directement. La version utilisable de ces codes se trouve dans la partie dédiée aux tests et à la présentation du code)*

## **A. Algorithmes**

### **1) Version brute**

Une première méthode de calcul consiste en une implémentation non-parallélisée du calcul.

**Algorithme brut**


**for** all $r_i$ in A:  
$\space\space$**for** all pairs ($a_{ij}$,$a_{ik}$) in $r_i$:  
$\space\space\space\space$ Emit ($c_i, c_j$) $→$ $a_{ij} a_{ik}$  
$\space\space$**end for**  
**end for**  
**Output** C




En pratique on utilise **Numpy** pour effectuer ce calcul "brut"


```
N = A.T.dot(A)
```

Comme évoqué dans l'introduction, cette implémentation n'est pas utilisable en pratique pour les matrices considérées (stockage de la matrice et temps de calcul).

### **2) Version parallèle naïve**

Une première version naïve faisant appel à un mapper et un reducer est proposée dans le papier. 

**Mapper naïf $(r_i)$**  
$\space\space\space\space$**for** all pairs ($a_{ij}$,$a_{ik}$) in $r_i$:  
$\space\space\space\space\space\space$ Emit ($c_i, c_j$) $→$ $a_{ij} a_{ik}$  
$\space\space\space\space$**end for**
  


---


**Reducer naïf $((c_i,c_j),<v_1, … , v_R>)$**  
$\space\space\space\space$**Output** $c_i^T c_j → ∑_{i = 1}^R v_i$

En pratique, nous avons utilisé **Numpy** pour implémenter simplement cette version

In [ ]:
def mapper(mat, norms_array, gamma):
    return mat.T@mat


def reducer(mat_list, norms_array, gamma):
    return sum(mat_list)

La manière naïve d'effectuer le calcul de $A^TA$ consiste à calculer tous les produits entre les colonnes de la matrice $A$ et à paralléliser le calcul.
On peut voir que l'implémentation naïve de MapReduce a une compléxité en $O(mL^2)$, ce qui est infaisable pour les matrices étudiées. 

### **3) Version parallèle efficace**

Pour un algorithme MapReduce, il existe deux sources principales de complexité:


*   une complexité liée aux communications entre les machines ("shuffle-size")
*   une complexité liée à la surcharge potentielle d'une matrice ("reduce-key")

Il est possible de réduire significativement les deux complexités avec l'algorithme DIMSUM proposé dans l'article. Dans ce cas, le reducer renvoie une variable aléatoire dont l'espérance est une normalisation des paramètres de $A^TA$. Il est en particulier possible de montrer que, pour un seuil ϵ donné, la complexité liée à la charge d'une machine ("reduce-key") est en $O(\frac{n^2}{ϵ^2})$, donc indépendante de $m$.

On choisit dans ce cas un $\gamma$, l'algorithme efficace s'écrit comme suit:

**Mapper DIMSUM $(r_i)$**  
$\space\space\space\space$**for** all pairs ($a_{ij}$,$a_{ik}$) in $r_i$:  
$\space\space\space\space\space\space$ With probability $min(1,\gamma \frac{1}{||c_j|| \cdot ||c_k||})$    
$\space\space\space\space\space\space\space\space\space\space$ Emit ($c_i, c_j$) $→$ $a_{ij} a_{ik}$  
$\space\space\space\space$**end for**
  


---


**Reducer DIMSUM $((c_i,c_j),<v_1, … , v_R>)$**  
$\space\space\space\space$**Output** $b_{ij} → \frac{1}{min(\gamma,||c_j|| \cdot ||c_k||)} ∑_{i = 1}^R v_i$

En voici, une version en Python

In [ ]:
def mapper(mat, norms_array, gamma):
    gamma_copy = gamma
    nrow, ncol = mat.shape
    output = np.zeros((ncol, ncol)) # note that ncol << nrow, so the for loops are OK
    for i_output in range(ncol):
        for j_output in range(ncol):
            # randomly choose pairs
            random_values = np.random.rand(nrow)
            probas = gamma_copy/(norms_array[i_output]*norms_array[j_output])*np.ones((nrow,))
            bool_vect = (probas < random_values)
            # sum chosen pairs
            output[i_output, j_output] = np.sum(mat[bool_vect, i_output]*mat[bool_vect, j_output])
    return output


def reducer(mat_list, norms_array, gamma):
    return 1/np.minimum(np.outer(norms_array, norms_array), gamma)*sum(mat_list)

## **B. Résultats théoriques**

A compléter

> Indented block



## **C. Technologies utilisées** 

Dans une logique d'optimisation de la performance de nos algorithmes, nous avons comparé l'usage de 2 structures de données différentes pour stocker les matrices: 


*   Stockage sous forme de **dictionnaires Python**
*   Stockage sous forme d'**arrays Numpy**



Comme suggéré dans l'article nous avons utilisé un algorithme MapReduce pour gérer en parallèle le grand nombre d'entrées de la matrice A. La très grande taille de la matrice (plusieurs gigaoctets) ainsi que la contrainte temporelle nous ont amenés à distribuer le calcul via les algorithmes présentés dans la partie I.A.

L'implémentation des algorithmes a été effectuée avec Python en faisant appel aux bibliothèques ```multirocessing``` et ```threading``` dont nous avons cherché à comparer les performances sur différentes matrices.



**Multiprocessing**  
Ce package permet le recours à des sous-process pour effectuer des calculs en parallèle. Il propose en particulier la fonction ```Pool``` pour gérer simplement les multiples sous-process utilisés



**Threading**  
Ce package permet de générer et de gérer différents threads pour effectuer les calculs en parallèle. 

La bibliothèque ```numpy``` a été utilisée pourn définir et manipuler les matrices plus simplement. Il est a noter que ```numpy``` fait déjà appel à des méthodes d'optimisation de la performance lors de calculs de matrices. Ce que nous avons pu confirmer en étudiant l'utilisation des différents coeurs de nos machines lors de l'appel à une version non-parallélisée du calcul. Pour résoudre ce problème et contrôler nous-même la performance, nous avons imposé à ```numpy``` de ne faire appel qu'à un seul coeur lors de l'execution, afin de contrôler la parallèlisation indépendemment.


```
# force numpy to use only a single processor, by changing the environment of the underlying libraries
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
import numpy as np
```





# **Conclusion**